In [53]:
%reload_ext autoreload
%autoreload 2

import nemo
import torch
import attrdict
import numpy as np

from ruamel import yaml
from torch.nn import functional as F
from nemo.collections import asr as nemo_asr

In [13]:
runner = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU,
)

[NeMo W 2020-04-19 21:21:06 deprecated:68] Function ``_get_trainer`` is deprecated. It is going to be removed in the future version.


In [14]:
MODEL_CONFIG = '/home/stanislavv/src/tts/NeMo/examples/asr/configs/quartznet15x5-libritts-durs.yaml'
yaml_loader = yaml.YAML(typ='safe')
with open(MODEL_CONFIG) as f:
    config = attrdict.AttrDict(yaml_loader.load(f))

config.sample_rate = SAMPLE_RATE
config.AudioToMelSpectrogramPreprocessor

AttrDict({'window_size': None, 'window_stride': None, 'n_window_size': 512, 'n_window_stride': 256, 'window': 'hann', 'normalize': 'per_feature', 'n_fft': 512, 'preemph': None, 'features': 64, 'lowfreq': 0, 'highfreq': None, 'log': True, 'log_zero_guard_type': 'clamp', 'log_zero_guard_value': 1e-05, 'dither': 0.0, 'pad_to': 16, 'frame_splicing': 1, 'stft_conv': True, 'pad_value': -11.52, 'mag_power': 2.0})

In [15]:
MAP = '/home/stanislavv/data/ljspeech/local/train.json'
SAMPLE_RATE = 22050
data_layer = nemo_asr.AudioToTextDataLayer(
    manifest_filepath=MAP,
    sample_rate=SAMPLE_RATE,
    labels=config.labels,
    batch_size=64,
    shuffle=False,
)

[NeMo I 2020-04-19 21:21:11 collections:144] Dataset loaded with 13036 files totalling 23.80 hours
[NeMo I 2020-04-19 21:21:11 collections:145] 0 files were filtered totalling 0.00 hours


In [16]:
a, al, t, tl = next(iter(data_layer._dataloader))
a.shape, al.shape

(torch.Size([64, 220061]), torch.Size([64]))

In [60]:
di = iter(data_layer._dataloader)
a1, *_ = next(di)
a2, *_ = next(di)
((torch.zeros_like(a1[:, :220061]) - a2[:, :220061]) ** 2).mean()

tensor(0.0032)

In [58]:
torch.zeros_like(a1[:, :220061]) - a2[:, :220061]

tensor([[ 9.1248e-03,  5.0964e-03,  3.2959e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.0071e-03, -9.4604e-04, -7.9346e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 6.7749e-03,  9.2468e-03,  7.7209e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-9.1553e-05, -6.1035e-05,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 3.0518e-05,  1.2207e-04,  1.5259e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.4414e-04,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [52]:
np.zeros_like(a1[:, :220061])

NameError: name 'np' is not defined

In [17]:
a1 = a[0]
a1.shape

torch.Size([220061])

In [44]:
def pad(x, to=16, dim=1, value=0):
    pads = [0, 0] * len(x.shape)
    pads[-(2 * dim + 1)] = (to - (x.shape[dim] % to)) % to
    return F.pad(x, pads, value=value)


x = torch.zeros((10, 10))
pad(x).shape

torch.Size([10, 16])

In [43]:
x = torch.zeros((10, 10))
pad(x).shape

1


torch.Size([16, 10])

In [37]:
first256 = a1[:256]
pad256 = pad(a1, to=256, dim=0).view(-1, 256)[0]
first256 - pad256

torch.Size([220061])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 